In [1]:
import pandas as pd # type: ignore
df_roster = pd.read_csv("2022-2023/22-23-Roster.csv", skiprows= [1]) #deletes unwanted repeated row of attributes
df_game_logs = pd.read_csv("2022-2023/22-23-game-logs.csv")
df_per_game = pd.read_csv("2022-2023/22-23-Per-Game.csv")
df_results = pd.read_csv("2022-2023/22-23-Results.csv")
df_team_n_opp = pd.read_csv("2022-2023/22-23-Team&Opp-Per-Game.csv")

In [2]:
#deletes 'Unnamed: 0' column from the dataframes
df_roster = df_roster.drop(df_roster.columns[0], axis = 1)
df_game_logs = df_game_logs.drop(df_game_logs.columns[0], axis = 1)
df_per_game = df_per_game.drop(df_per_game.columns[0], axis = 1)
df_results = df_results.drop(df_results.columns[0], axis = 1)
df_team_n_opp = df_team_n_opp.drop(df_team_n_opp.columns[0], axis = 1)

In [3]:
#goal: clean up df_roster and make it useable by converting string values into integers as appropriate 
df_rosterc = df_roster.drop(['RSCI Top 100', 'High School', 'Hometown'], axis = 1) #delete RSCI Top 100 col because unneeded 
df_rosterc = df_rosterc.join(df_per_game[['PTS','AST']]) #combines columns from df_per_game as they are the same as some of the summary 
def extract_rebounds(summary):
    if 'Reb' in summary:
        # Split by commas and find the part containing 'Reb'
        parts = summary.split(', ')
        for part in parts:
            if 'Reb' in part:
                return float(part.split(' ')[0])  # Extract the number before 'Reb'
    return 0.0  # Return 0.0 if no 'Reb' is found
df_rosterc['REB'] = df_rosterc['Summary'].apply(extract_rebounds)
df_rosterc



,Player,#,Class,Pos,Height,Weight,Summary,PTS,AST,REB
0,Terrence Shannon,0,SR,G,6-6,210.0,"17.2 Pts, 4.6 Reb, 2.8 Ast",17.2,2.8,4.6
1,Matthew Mayer,24,SR,G,6-9,225.0,"12.5 Pts, 5.3 Reb, 1.1 Ast",12.5,1.1,5.3
2,Coleman Hawkins,33,JR,F,6-10,200.0,"9.9 Pts, 6.3 Reb, 3.0 Ast",9.9,3.0,6.3
3,Dain Dainja,42,SO,F,6-9,270.0,"9.5 Pts, 5.5 Reb, 0.5 Ast",9.5,0.5,5.5
4,Jayden Epps,3,FR,G,6-2,190.0,"9.5 Pts, 1.8 Reb, 1.5 Ast",9.5,1.5,1.8
5,RJ Melendez,15,SO,G,6-7,205.0,"6.0 Pts, 3.5 Reb, 0.9 Ast",7.0,2.1,3.5
6,Sencire Harris,1,FR,G,6-4,160.0,"3.7 Pts, 2.1 Reb, 0.9 Ast",6.0,0.9,2.1
7,Ty Rodgers,20,FR,G,6-6,200.0,"3.3 Pts, 4.0 Reb, 1.0 Ast",4.0,0.4,4.0
8,Skyy Clark,55,FR,G,6-3,200.0,"7.0 Pts, 3.7 Reb, 2.1 Ast",3.7,0.9,3.7
9,Luke Goode,10,SO,G,6-7,200.0,"4.0 Pts, 1.6 Reb, 0.4 Ast",3.3,1.0,1.6


In [4]:
df_per_game #all values are good as is, potentially look into removing nan values but as it stands, it's alright

,Rk,Player,G,GS,MP,FG,FGA,FG%,2P,2PA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Terrence Shannon,31,30,32.1,5.1,11.6,0.442,3.5,6.6,...,0.790,1.2,3.4,4.6,2.8,1.3,0.5,2.6,2.4,17.2
1,2,Matthew Mayer,33,33,26.6,4.0,9.9,0.407,2.0,3.8,...,0.741,1.0,4.4,5.3,1.1,0.8,1.2,1.4,2.4,12.5
2,3,Coleman Hawkins,33,33,32.5,3.7,8.5,0.441,2.6,4.5,...,0.614,1.9,4.3,6.3,3.0,1.1,1.2,2.5,1.8,9.9
3,4,Dain Dainja,33,21,20.5,4.0,6.2,0.636,4.0,6.2,...,0.525,2.0,3.5,5.5,0.5,0.8,1.2,1.5,1.9,9.5
4,5,Jayden Epps,31,11,24.6,3.6,8.6,0.414,2.4,4.7,...,0.778,0.2,1.6,1.8,1.5,0.6,0.2,1.3,1.5,9.5
5,6,Skyy Clark,13,12,24.5,2.3,5.6,0.411,1.2,2.2,...,0.696,0.5,3.2,3.7,2.1,0.5,0.2,2.1,2.8,7.0
6,7,RJ Melendez,32,18,21.5,2.0,5.3,0.376,1.3,2.5,...,0.833,1.0,2.5,3.5,0.9,0.7,0.4,0.9,1.7,6.0
7,8,Luke Goode,10,0,13.9,1.5,3.1,0.484,0.7,1.2,...,1.000,0.4,1.2,1.6,0.4,0.5,0.4,0.2,1.7,4.0
8,9,Sencire Harris,33,7,14.5,1.6,3.7,0.430,1.2,2.4,...,0.200,0.6,1.5,2.1,0.9,0.8,0.3,0.6,1.5,3.7
9,10,Ty Rodgers,33,0,17.4,1.5,2.8,0.516,1.5,2.8,...,0.387,1.6,2.4,4.0,1.0,0.7,0.2,0.8,1.8,3.3
